In [ ]:
import pandas as pd
import numpy as np

df_for_revenue = pd.read_csv('/Users/jacob.perius/psa_segment_testing/incrementality_testing/final_groups/final_assignments.csv')
df_for_revenue

In [ ]:
dma_df = pd.read_csv('/Users/jacob.perius/psa_segment_testing/incrementality_testing/data/ENV _ Census _ ZIP to DMA.csv')

dma_df.head(5)

In [ ]:
dma_df['zip_code'] = dma_df['zip_code'].astype('str').str.zfill(5)

In [ ]:
google_df = pd.read_csv('/Users/jacob.perius/psa_segment_testing/incrementality_testing/data/google_ads_dma_features.csv', low_memory=False)

google_df.head(70)

In [ ]:
google_df['Clicks'].describe()

In [ ]:
google_df['CTR'] = google_df['Clicks'].str.replace(',', '').astype('int64') / google_df['Impr.'].str.replace(',', '').astype('int64')

google_df

In [ ]:
import re

google_df = google_df.iloc[:-6].copy()

google_df.loc[:, 'zip_code'] = google_df['Matched location'].apply(lambda x: re.search(r'^\d{5}', x).group(0))

#google_df.loc[:, 'All conv. value'] = google_df['All conv. value'].str.replace(',', '').str.strip().astype('float64')
google_df = google_df.drop('Matched location', axis=1)
google_df

In [ ]:
google_dma_df = pd.merge(google_df, dma_df, on='zip_code', how='right')[['dma_code', 'dma_description', 'All conv. value']]

google_dma_df

In [ ]:
grouped_google_dma_df = google_dma_df.groupby(['dma_code', 'dma_description']).mean().reset_index()

grouped_google_dma_df

In [ ]:
merged_df = pd.merge(df_for_revenue, grouped_google_dma_df, on=['dma_code', 'dma_description'], how='left')

merged_df

In [ ]:
group_a_df = merged_df[merged_df['group'] == 'Group A']

print(group_a_df['CTR'].mean())

group_a_df

In [ ]:
group_b_df = merged_df[merged_df['group'] == 'Group B']

print(group_b_df['CTR'].mean())

group_b_df

In [ ]:
import json
import plotly.express as px

with open('/Users/jacob.perius/psa_segment_testing/neilsen-dma-markets-albers-projection_1356.geojson', 'r') as f:
    dma_geojson_str = f.read()

# First parse to remove the outer string layer
dma_geojson = json.loads(dma_geojson_str)

# Second parse if needed (in case it's double-encoded)
if isinstance(dma_geojson, str):
    dma_geojson = json.loads(dma_geojson)

fig = px.choropleth(
    names_and_groups_df,
    geojson=dma_geojson,
    locations='dma_code',
    color='group',
    featureidkey='properties.dma_code',
    color_discrete_map={
        'Group A': 'blue',
        'Group B': 'orange',
        'Group C': 'green'
    },
    hover_data={'dma_code': True, 'dma_description': True, 'group': True}
)

fig.update_traces(marker_line_width=1, marker_opacity=1.0)

fig.update_geos(
    fitbounds="locations",
    visible=False,
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                legend_title_text='Test Group'
                )

fig.show()